In [25]:
import pandas as pd

In [26]:
import os
import pathlib

root_dir = pathlib.PurePath(os.path.dirname(os.getcwd())).parent
data_dir = os.path.join(root_dir, 'data')

In [27]:
import nest_asyncio

nest_asyncio.apply()

# LLM 기반 질의 생성 실습

실습을 진행하기 위해, 먼저 Corpus 데이터셋을 준비해야 합니다. 
이번 실습에서는, 전 실습에서 진행했던 Corpus 파일을 그대로 활용해 봅시다!

병욱 강사님이 진행하신 Semantic Chunking에서 나왔던 Corpus 파일을 활용하겠습니다.

In [28]:
# 데이터셋 로드 및 구경
corpus_data_path = os.path.join(data_dir, 'data_creation', 'corpus.parquet')
corpus_df = pd.read_parquet(corpus_data_path, engine='pyarrow')

In [29]:
corpus_df.head()

,doc_id,contents,path,start_end_idx,metadata
0,136df061-4be7-4809-9a69-cc0fa6fdc0e2,# 중소벤처기업부공고제2024–300호# 2024년 글로벌팁스(GlobalTIPS)...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 0]","{'last_modified_datetime': '2024-12-11', 'next..."
1,2ade0a0c-96f3-4623-800f-028da810883a,"□ 협약기간: 협약 시작일로부터 10개월 이내* 사업기간은 총 3년으로 선정 후, ...",/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 0]","{'last_modified_datetime': '2024-12-11', 'next..."
2,a8eaed13-4383-45fa-a774-73f5e54b4a95,# 서류및발표평가# 결과에최대 5점까지 가점부여|가점세부항목|점수|비고|\n|---...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 0]","{'last_modified_datetime': '2024-12-11', 'next..."
3,e0341a6a-4135-4cf1-8bd0-dfceeb052bc2,※세부일정은대내‧외사정에의해변경될수있음|공고|창업기업신청‧접수|요건검토및선정평가|\...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 0]","{'last_modified_datetime': '2024-12-11', 'next..."
4,228d107f-edcc-49df-93ce-2e9d64d51c29,# 6 유의사항◈창업지원사업에신청하는창업기업은관련법령에따라 다음사항에유의하여야합니다...,/Users/kimbwook/PycharmProjects/fast-campus/no...,"[0, 0]","{'last_modified_datetime': '2024-12-11', 'next..."


In [30]:
corpus_df.iloc[0]

doc_id                        136df061-4be7-4809-9a69-cc0fa6fdc0e2
contents         # 중소벤처기업부공고제2024–300호# 2024년 글로벌팁스(GlobalTIPS)...
path             /Users/kimbwook/PycharmProjects/fast-campus/no...
start_end_idx                                               [0, 0]
metadata         {'last_modified_datetime': '2024-12-11', 'next...
Name: 0, dtype: object

In [31]:
corpus_df.iloc[0]['contents']

'# 중소벤처기업부공고제2024–300호# 2024년 글로벌팁스(GlobalTIPS) 창업기업 모집 공고해외투자와 연계하여 유망 창업기업의 해외 진출 및 글로벌 기업으로의 성장을 지원하는 『2024년 글로벌팁스(GlobalTIPS)』에 참여할 창업기업을 다음과 같이 모집합니다.2024년 5월 7일중소벤처기업부장관# 1 사업개요□ 사업목적: 미래 신성장 동력 발굴과 글로벌 선도 기업 육성 등을 위해 해외 벤처 캐피탈 등으로부터 투자받은 유망 창업기업의 글로벌 진출 활성화 도모□ 지원대상: 해외 VC로부터 20만 불 이상 투자를 유치하고 해외 법인 설립 희망하는 업력 7년 이내 창업기업*「중소기업 창업 지원법」에 따른 신산업 분야 ([붙임4] 참조) 창업기업인 경우 업력 10년 기업까지 지원 가능□ 지원내용: 사업화 자금 (최대 2억원, 평균 1.8억원) 현지 진출 프로그램 등|사업화 자금|현지 진출 프로그램|\n|---|---|\n|시제품 제작, 지재권 취득, 사업모델 (BM) 개선 등에 소요되는 비용 지원|실무 교육, 멘토링, 글로벌 네트워킹, 후속 투자 유치 등 주관 기관의 창업 프로그램 운영|□ 선정규모: 20개사 내외'

AutoRAG를 통하여 Corpus를 생성하지 않았더라도, 유사하게 만들어주면 됩니다!

In [32]:
from dotenv import load_dotenv

load_dotenv()

True

## 전체 QA creation 돌려보기

In [33]:
from autorag.data.qa.schema import Corpus
from llama_index.llms.openai import OpenAI

from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import (
    make_basic_gen_gt,
    make_concise_gen_gt,
)
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

llm = OpenAI()
initial_corpus = Corpus(corpus_df)
initial_qa = (
    initial_corpus.sample(random_single_hop, n=3) # sample only three
    .map(
        lambda df: df.reset_index(drop=True),
    )
    .make_retrieval_gt_contents()
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,
		lang='ko',
    )
    .batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,
		lang='ko',
    )
    .batch_apply(
        make_concise_gen_gt,  # answer generation (concise)
        llm=llm,
		lang='ko',
    )
    .filter(
        dontknow_filter_rule_based,  # filter don't know
        lang="ko",
    )
)

[01/12/25 15:46:19] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=177122;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=39613;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=171559;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=77023;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=430461;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=52260;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/12/25 15:46:20] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=269707;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=51638;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=906150;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=233411;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=951098;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=830427;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/12/25 15:46:21] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=170991;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=409393;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=941285;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=271218;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=665017;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=750900;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [34]:
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,88532600-12ba-45f4-b135-12d3bc7c1f75,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]],"[[본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점...",해외VC가 투자 유치 조건으로 요구하는 최소 금액은 얼마입니까?,"[해외VC가 투자 유치 조건으로 요구하는 최소 금액은 20만불입니다., 20만불]"
1,8580027f-7219-4f7c-8684-f9c3be208886,[[dedda772-b75a-4cb5-af8d-63150601610a]],"[[1. ①팁스성공기업 ,시드포함)(프리\n2. ②글로벌지원사업(글로벌액셀러레이팅,...",해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까?,[해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니다. 텍스트에서...
2,6ce1d2e4-3ca7-491f-9a26-4999f3b6d785,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]],[[# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표...,창업기업 선정 평가는 총 몇 단계로 이루어지나요?,"[창업기업 선정 평가는 총 4단계로 이루어집니다. 서류평가, 발표평가, 최종선정자격..."


## 1. Retrieval gt 선정 (랜덤)

In [39]:
initial_corpus = Corpus(corpus_df)
initial_qa = initial_corpus.sample(random_single_hop, n=3) # sample only three

In [40]:
initial_qa.data

,qid,retrieval_gt
0,54a4c63a-4aba-4d00-b1f8-0ba6a489d8be,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]]
1,e54e2d0b-ad31-4660-a760-f9b60f2162b4,[[dedda772-b75a-4cb5-af8d-63150601610a]]
2,d47aaa3d-67c5-491b-8060-c46eca82f85d,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]]


Corpus 데이터셋에서 각 retrieval_gt id에 해당하는 데이터를 가져옵니다.

In [41]:
initial_qa = (initial_qa.map(
        lambda df: df.reset_index(drop=True),
    )
    .make_retrieval_gt_contents())

In [42]:
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents
0,54a4c63a-4aba-4d00-b1f8-0ba6a489d8be,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]],"[[본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점..."
1,e54e2d0b-ad31-4660-a760-f9b60f2162b4,[[dedda772-b75a-4cb5-af8d-63150601610a]],"[[1. ①팁스성공기업 ,시드포함)(프리\n2. ②글로벌지원사업(글로벌액셀러레이팅,..."
2,d47aaa3d-67c5-491b-8060-c46eca82f85d,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]],[[# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표...


## 2. 다양한 LLM 질의 생성

- factoid_query_gen: Factoid 질문 생성
- concept_completion_query_gen: Concept Completion 질문 생성

In [43]:
factoid_qa = initial_qa.batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,
		lang='ko',
    )

[01/12/25 15:48:41] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=285133;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=340332;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=290969;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=104556;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=993023;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=770223;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [44]:
factoid_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query
0,54a4c63a-4aba-4d00-b1f8-0ba6a489d8be,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]],"[[본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점...",해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마인가요?
1,e54e2d0b-ad31-4660-a760-f9b60f2162b4,[[dedda772-b75a-4cb5-af8d-63150601610a]],"[[1. ①팁스성공기업 ,시드포함)(프리\n2. ②글로벌지원사업(글로벌액셀러레이팅,...",해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까?
2,d47aaa3d-67c5-491b-8060-c46eca82f85d,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]],[[# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표...,창업기업 선정평가의 평가절차는 몇 단계로 이루어져 있나요?


In [45]:
factoid_qa.data['query']

0    해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마인가요?
1      해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까?
2             창업기업 선정평가의 평가절차는 몇 단계로 이루어져 있나요?
Name: query, dtype: object

In [46]:
from autorag.data.qa.query.llama_gen_query import concept_completion_query_gen

concept_completion_qa = initial_qa.batch_apply(
	    concept_completion_query_gen,  # query generation
        llm=llm,
        lang='ko',
    )

[01/12/25 15:50:15] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=277497;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=193223;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=40021;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=627287;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=484248;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=622235;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [47]:
concept_completion_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query
0,54a4c63a-4aba-4d00-b1f8-0ba6a489d8be,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]],"[[본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점...",해외VC가 투자사로 신청할 수 있는 조건은 무엇인가요?
1,e54e2d0b-ad31-4660-a760-f9b60f2162b4,[[dedda772-b75a-4cb5-af8d-63150601610a]],"[[1. ①팁스성공기업 ,시드포함)(프리\n2. ②글로벌지원사업(글로벌액셀러레이팅,...",해외VC투자를 받기 위해 제출해야 하는 서류는 무엇인가요?
2,d47aaa3d-67c5-491b-8060-c46eca82f85d,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]],[[# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표...,창업기업 선정을 위한 평가 절차는 어떻게 되나요?


In [48]:
concept_completion_qa.data['query']

0      해외VC가 투자사로 신청할 수 있는 조건은 무엇인가요?
1    해외VC투자를 받기 위해 제출해야 하는 서류는 무엇인가요?
2         창업기업 선정을 위한 평가 절차는 어떻게 되나요?
Name: query, dtype: object

## 3. 질문 Evolving

In [52]:
from autorag.data.qa.evolve.llama_index_query_evolve import reasoning_evolve_ragas

# Reasoning Evolving

reasoning_qa = factoid_qa.batch_apply(
	reasoning_evolve_ragas,
	llm=llm,
	lang='ko',
)


[01/12/25 16:19:10] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=369920;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=763290;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=637598;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=44697;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=492767;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=924859;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [53]:
reasoning_qa.data['query']

0            해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마인가요?
1    해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보는 5번 항목에 있습니다....
2                             창업기업 선정평가는 몇 단계로 이루어지나요?
Name: query, dtype: object

In [54]:
# Conditional Evolving

from autorag.data.qa.evolve.llama_index_query_evolve import conditional_evolve_ragas

conditional_qa = factoid_qa.batch_apply(
	conditional_evolve_ragas,
    llm=llm,
    lang='ko',
)


[01/12/25 16:29:40] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=967553;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=835519;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=884745;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=450316;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=706104;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=901828;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [55]:
conditional_qa.data['query']

0    해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마이며, 어떤 조건...
1    해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까? 해당 정보는...
2                     창업기업 선정평가의 평가절차는 몇 단계로 이루어져 있나요?
Name: query, dtype: object

In [56]:
# Compress Evolving

from autorag.data.qa.evolve.llama_index_query_evolve import compress_ragas

compress_qa = factoid_qa.batch_apply(
	compress_ragas,
    llm=llm,
    lang='ko',
)

[01/12/25 16:30:29] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=341758;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=336903;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=426753;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=756563;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=689958;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=998967;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [57]:
compress_qa.data['query']

0     해외VC 투자를 받기 위한 최소 투자금은 얼마인가요?
1    해외VC가 국내에 사무실을 갖고 있는지 알고 계신가요?
2    창업기업 선정평가의 평가과정은 몇 단계로 되어 있나요?
Name: query, dtype: object

## 4. 답변 생성

In [58]:
qa_and_answer = conditional_qa.batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,
		lang='ko',
    )

[01/12/25 16:32:03] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=811232;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=55371;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=30639;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=403204;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/12/25 16:32:04] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=481545;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=23278;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [59]:
qa_and_answer.data[['query', 'generation_gt']]

,query,generation_gt
0,"해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마이며, 어떤 조건...",[해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 20만불(USD) ...
1,해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까? 해당 정보는...,[해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보는 주어진 Text에서 ...
2,창업기업 선정평가의 평가절차는 몇 단계로 이루어져 있나요?,"[창업기업 선정평가의 평가절차는 총 4단계로 이루어져 있습니다. (1. 요건검토, ..."


In [60]:
qa_and_answer = qa_and_answer.batch_apply(
	make_concise_gen_gt,
	llm=llm,
    lang='ko',
)

[01/12/25 16:32:46] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=799955;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=719808;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=87634;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=333885;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=645500;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=569941;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [61]:
qa_and_answer.data[['query', 'generation_gt']]

,query,generation_gt
0,"해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마이며, 어떤 조건...",[해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 20만불(USD) ...
1,해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까? 해당 정보는...,[해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보는 주어진 Text에서 ...
2,창업기업 선정평가의 평가절차는 몇 단계로 이루어져 있나요?,"[창업기업 선정평가의 평가절차는 총 4단계로 이루어져 있습니다. (1. 요건검토, ..."


In [62]:
qa_and_answer.data.iloc[2]['generation_gt']

['창업기업 선정평가의 평가절차는 총 4단계로 이루어져 있습니다. (1. 요건검토, 2. 서류평가, 3. 발표평가, 4. 최종선정자격기준검토및 서류평가대상자 확정)',
 '4단계 (요건검토, 서류평가, 발표평가, 최종선정자격기준검토)']

## 5. Filtering

In [66]:
final_qa = qa_and_answer.filter(
        dontknow_filter_rule_based,  # filter don't know
        lang="ko",
    ).map(
        lambda df: df.reset_index(drop=True),
    )

In [67]:
final_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,54a4c63a-4aba-4d00-b1f8-0ba6a489d8be,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]],"[[본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점...","해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마이며, 어떤 조건...",[해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 20만불(USD) ...
1,e54e2d0b-ad31-4660-a760-f9b60f2162b4,[[dedda772-b75a-4cb5-af8d-63150601610a]],"[[1. ①팁스성공기업 ,시드포함)(프리\n2. ②글로벌지원사업(글로벌액셀러레이팅,...",해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까? 해당 정보는...,[해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보는 주어진 Text에서 ...
2,d47aaa3d-67c5-491b-8060-c46eca82f85d,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]],[[# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표...,창업기업 선정평가의 평가절차는 몇 단계로 이루어져 있나요?,"[창업기업 선정평가의 평가절차는 총 4단계로 이루어져 있습니다. (1. 요건검토, ..."


In [68]:
from autorag.data.qa.filter.dontknow import dontknow_filter_llama_index

final_qa = qa_and_answer.batch_filter(
	dontknow_filter_llama_index,
	llm=llm,
	lang='ko',
).map(
        lambda df: df.reset_index(drop=True),
    )

[01/12/25 16:35:19] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=701008;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=834714;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=594792;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=888469;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[01/12/25 16:35:20] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=972928;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=681874;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=603148;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=441744;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=20893;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=406754;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=618364;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=584578;file:///Users/jeffrey/PycharmProjects/fast-campus/venv/lib/python3.10/site-packages/httpx/_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [69]:
final_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,54a4c63a-4aba-4d00-b1f8-0ba6a489d8be,[[b6b2b214-5886-48d4-86a5-9286a0953bd3]],"[[본점과 지점이 있는 경우 본점 사업자로만 신청 가능하며, 중복 신청한 경우 본점...","해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 얼마이며, 어떤 조건...",[해외VC가 투자유치 조건을 충족하기 위해 필요한 최소 투자금은 20만불(USD) ...
1,e54e2d0b-ad31-4660-a760-f9b60f2162b4,[[dedda772-b75a-4cb5-af8d-63150601610a]],"[[1. ①팁스성공기업 ,시드포함)(프리\n2. ②글로벌지원사업(글로벌액셀러레이팅,...",해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보가 있습니까? 해당 정보는...,[해외VC가 국내지사나 사무소를 보유하고 있는지에 대한 정보는 주어진 Text에서 ...
2,d47aaa3d-67c5-491b-8060-c46eca82f85d,[[f5c0a9ee-4d4e-4d28-90cc-9e5ec615c42e]],[[# 5 평가및선정□평가절차:총2단계평가(서류 + (현장실사))를통해최종선정 발표...,창업기업 선정평가의 평가절차는 몇 단계로 이루어져 있나요?,"[창업기업 선정평가의 평가절차는 총 4단계로 이루어져 있습니다. (1. 요건검토, ..."
